# Insalling packages

In [ ]:
!sudo apt install trimmomatic

In [41]:
!sudo apt-get install bwa
!sudo apt-get install samtools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  samtools
The following NEW packages will be installed:
  bwa
0 upgraded, 1 newly installed, 0 to remove and 30 not upgraded.
Need to get 176 kB of archives.
After this operation, 404 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 bwa amd64 0.7.17-1ubuntu0.1 [176 kB]
Fetched 176 kB in 1s (342 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package bwa.
(Reading database ... 162

In [5]:
!wget http://cab.spbu.ru/files/release3.10.1/SPAdes-3.10.1-Linux.tar.gz
!tar -xzf SPAdes-3.10.1-Linux.tar.gz

--2021-04-04 16:54:53--  http://cab.spbu.ru/files/release3.10.1/SPAdes-3.10.1-Linux.tar.gz
Resolving cab.spbu.ru (cab.spbu.ru)... 195.70.219.98
Connecting to cab.spbu.ru (cab.spbu.ru)|195.70.219.98|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10294085 (9.8M) [application/octet-stream]
Saving to: ‘SPAdes-3.10.1-Linux.tar.gz’

SPAdes-3.10.1-Linux 100%[===================>]   9.82M  7.19MB/s    in 1.4s    

2021-04-04 16:54:54 (7.19 MB/s) - ‘SPAdes-3.10.1-Linux.tar.gz’ saved [10294085/10294085]



# Aligning raw reads

In [37]:
!bwa index MG1655-K12.first400K.fasta
!bwa mem MG1655-K12.first400K.fasta ecoli_400K_err_1.fastq.gz ecoli_400K_err_2.fastq.gz > alignment.sam
!samtools view -S -b alignment.sam > alignment.bam
!samtools sort alignment.bam > alignment_sorted.bam
!samtools index alignment_sorted.bam

[bwa_index] Pack FASTA... 0.01 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.09 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.04 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index MG1655-K12.first400K.fasta
[main] Real time: 0.227 sec; CPU: 0.196 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 100000 sequences (10000000 bp)...
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48301, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.27, 10.16)
[M::mem_pestat] low and high boundaries for proper pairs: (1

In [38]:
!samtools flagstat alignment_sorted.bam

2763204 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2762617 + 0 mapped (99.98% : N/A)
2763204 + 0 paired in sequencing
1381602 + 0 read1
1381602 + 0 read2
2760820 + 0 properly paired (99.91% : N/A)
2762132 + 0 with itself and mate mapped
485 + 0 singletons (0.02% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


# Aligning SPADES result 

In [39]:
!python3.5 ./SPAdes-3.10.1-Linux/bin/spades.py --only-error-correction --pe1-1 ecoli_400K_err_1.fastq.gz --pe1-2 ecoli_400K_err_2.fastq.gz -o corrected_spades

Command line: ./SPAdes-3.10.1-Linux/bin/spades.py	--only-error-correction	--pe1-1	/content/ecoli_400K_err_1.fastq.gz	--pe1-2	/content/ecoli_400K_err_2.fastq.gz	-o	/content/corrected_spades	

System information:
  SPAdes version: 3.10.1
  Python version: 3.5.10
  OS: Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic

Output dir: /content/corrected_spades
Mode: ONLY read error correction (without assembling)
Debug mode is turned OFF

Dataset parameters:
  Multi-cell mode (you should set '--sc' flag if input data was obtained with MDA (single-cell) technology or --meta flag if processing metagenomic dataset)
  Reads:
    Library number: 1, library type: paired-end
      orientation: fr
      left reads: ['/content/ecoli_400K_err_1.fastq.gz']
      right reads: ['/content/ecoli_400K_err_2.fastq.gz']
      interlaced reads: not specified
      single reads: not specified
Read error correction parameters:
  Iterations: 1
  PHRED offset will be auto-detected
  Corrected reads will be compressed

In [40]:
!bwa index MG1655-K12.first400K.fasta
!bwa mem MG1655-K12.first400K.fasta corrected_spades/corrected/ecoli_400K_err_1.fastq.00.0_0.cor.fastq.gz corrected_spades/corrected/ecoli_400K_err_2.fastq.00.0_0.cor.fastq.gz > alignment_spades.sam
!samtools view -S -b alignment_spades.sam > alignment_spades.bam
!samtools sort alignment_spades.bam > alignment_spades_sorted.bam
!samtools index alignment_spades_sorted.bam

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.09 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.04 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index MG1655-K12.first400K.fasta
[main] Real time: 0.211 sec; CPU: 0.192 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 100000 sequences (10000000 bp)...
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48352, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.27, 10.15)
[M::mem_pestat] low and high boundaries for proper pairs: (1

In [41]:
!samtools flagstat alignment_spades_sorted.bam

2716052 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2715942 + 0 mapped (100.00% : N/A)
2716052 + 0 paired in sequencing
1358026 + 0 read1
1358026 + 0 read2
2714584 + 0 properly paired (99.95% : N/A)
2715874 + 0 with itself and mate mapped
68 + 0 singletons (0.00% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


# Aligning Trimmomatic results

In [42]:
!TrimmomaticPE -phred33 ecoli_400K_err_1.fastq.gz ecoli_400K_err_2.fastq.gz -baseout corrected_trim LEADING:20 TRAILING:20  SLIDINGWINDOW:10:20 MINLEN:20

TrimmomaticPE: Started with arguments:
 -phred33 ecoli_400K_err_1.fastq.gz ecoli_400K_err_2.fastq.gz -baseout corrected_trim LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:20
Multiple cores found: Using 2 threads
Using templated Output files: corrected_trim_1P corrected_trim_1U corrected_trim_2P corrected_trim_2U
Input Read Pairs: 1381602 Both Surviving: 1344536 (97.32%) Forward Only Surviving: 23598 (1.71%) Reverse Only Surviving: 10815 (0.78%) Dropped: 2653 (0.19%)
TrimmomaticPE: Completed successfully


In [43]:
!bwa mem MG1655-K12.first400K.fasta corrected_trim_1P corrected_trim_2P > alignment_trim.sam
!samtools view -S -b alignment_trim.sam > alignment_trim.bam
!samtools sort alignment_trim.bam > alignment_trim_sorted.bam
!samtools index alignment_trim_sorted.bam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 110200 sequences (10000110 bp)...
[M::process] read 110814 sequences (10000040 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 53018, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.25, 10.17)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 110200 reads in 3.044 CPU sec, 3.000 real sec
[M::process] read 111006 sequences (10000050 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 53373, 0, 0)
[M::mem_pestat] skip orientat

In [44]:
!samtools flagstat alignment_trim_sorted.bam

2689072 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2687824 + 0 mapped (99.95% : N/A)
2689072 + 0 paired in sequencing
1344536 + 0 read1
1344536 + 0 read2
2685506 + 0 properly paired (99.87% : N/A)
2686782 + 0 with itself and mate mapped
1042 + 0 singletons (0.04% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


# Counting errors

In [45]:
!pip install pysam
!pip install tqdm
import pysam
from tqdm.notebook import tqdm
import pandas as pd

Для каждого рида из исправленного и обычного выравнивания будем строить словари, где ключ -- имя рида, значение -- сам рид.
После этого проходимся по ним, ищем риды, у которых ID совпадают, для них уже смотрим соответствие (также нахоим этот кусок в референсе).
В сам ответ записываем просто сравнивая позицию в референсе с позицией в старом и новом выравниваниях. 

In [46]:
def count_errors(samfile_raw: pysam.AlignmentFile, samfile_corrected: pysam.AlignmentFile):
  errors = [[0 for _ in range(3)] for _ in range(2)]
  raw_dict = {}
  for read in samfile_raw.fetch():
    if not read.is_unmapped:
      raw_dict[read.query_name] = read
  print("created raw dictionary")

  corrected_dict = {}
  for read in samfile_corrected.fetch():
    if not read.is_unmapped:
      corrected_dict[read.query_name] = read
  print("created corrected dictionary")

  for read_name, read_raw in tqdm(raw_dict.items()):
    dict_raw = {ref_i: None if read_i is None else read_raw.query_sequence[read_i] \
                                   for read_i, ref_i in read_raw.get_aligned_pairs()}
    if read_name in corrected_dict:
      read_corr = corrected_dict[read_name]
      dict_corr = {ref_i: None if read_i is None else read_corr.query_sequence[read_i] \
                                   for read_i, ref_i in read_corr.get_aligned_pairs()}
      dict_ref = {}
      for read_i, ref_i, ref_l in read_raw.get_aligned_pairs(with_seq = True):
        dict_ref[ref_i] = ref_l
      for index in dict_raw:
        raw_value = dict_raw[index]
        cor_value = dict_corr[index] if index in dict_corr else None
        ref_value = dict_ref[index] if index in dict_ref else None
        if cor_value == 'N':
          errors[int(ref_value == raw_value)][2] += 1
        else:
          errors[int(ref_value == raw_value)][int(ref_value == cor_value)] += 1
  return pd.DataFrame(errors)

In [47]:
print('PROCESSING THE TRIMMOMATIC RESULTS')
samfile_trim = pysam.AlignmentFile("alignment_trim_sorted.bam", "rb")
samfile = pysam.AlignmentFile("alignment_sorted.bam", "rb")
print(count_errors(samfile_raw=samfile, samfile_corrected=samfile_trim))

PROCESSING THE TRIMMOMATIC RESULTS
created raw dictionary
created corrected dictionary



         0          1      2
0  1006661     134359  68596
1  9539618  121291214      1


In [48]:
print('PROCESSING THE SPADES RESULTS')
samfile_trim = pysam.AlignmentFile("alignment_spades_sorted.bam", "rb")
samfile = pysam.AlignmentFile("alignment_sorted.bam", "rb")
print(count_errors(samfile_raw=samfile, samfile_corrected=samfile_trim))

PROCESSING THE SPADES RESULTS
created raw dictionary
created corrected dictionary



         0          1       2
0  1087323       5357  144007
1  7816149  124124489  189747
